In [ ]:
import os
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.types import StructType, StructField, StringType
from delta import *

warehouse_diretory_path = '[YOUR_WAREHOUSE]'
hgnc_gene_annotation_file = 'file:///[FILE_FULL_NAME:hgnc_complete_set.txt]'
biological_database_name = 'biological_database'
hgnc_gene_annotation_table_name = 'bronze_hgnc_gene_annotation'

conf = SparkConf()
conf.setAll(
    [
        ('spark.master', 'local[*]'), 
        ('spark.driver.host', 'localhost'),
        ('spark.app.name', 'HGNC Gene Annotation Importer'),
        ('spark.ui.showConsoleProgress', 'true'),
        ('spark.sql.execution.arrow.pyspark.enabled', 'true'),         
        ('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension'),
        ('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog'),
        ('spark.sql.warehouse.dir', warehouse_diretory_path),
        ('spark.driver.extraJavaOptions', f'-Dderby.system.home={warehouse_diretory_path}')
    ])

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

In [ ]:
hgnc_gene_annotation_df = spark.read \
    .option('header', True) \
    .options(delimiter='\t') \
    .csv(hgnc_gene_annotation_file)

In [ ]:
spark.sql(f'CREATE DATABASE IF NOT EXISTS {biological_database_name};')

In [ ]:
spark.sql(f'USE {biological_database_name};')

In [ ]:
hgnc_gene_annotation_df.write \
    .format('delta') \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .option('partitionOverwriteMode', 'dynamic') \
    .saveAsTable(hgnc_gene_annotation_table_name)

In [ ]:
spark.stop()